## Who Are They?

### This project will be scraping images or actors and actresses from Google Images to create a dataset that will then be used in a facial recognition project to identify actors and actresses in movies. This notebook only deals with the collection and labling of faces and the building of the dataset. The data will be stored in an S3 bucket.


### *By: Connor Secen*

### Setup:
#### This is the initial setup of the notebook, importing all the packages needed and setting main variables.

In [1]:
# all required imports
from bs4 import BeautifulSoup
from selenium import webdriver
from PIL import Image
import shutil
import hashlib
import multiprocessing
import requests
import cv2
import config
import boto3
import time
import os
import io

In [2]:
s3 = boto3.client('s3')   # instance of aws S3

# preset variables 
max_results = 350   # number of images collected
actor_num = 4   # represents number of pages == (n-1)*100 actors
group_size = 5   # number of actors images collected at once
imdb_url = 'https://www.imdb.com/list/ls058011111/?page={0}'   # imdb top 1000 actors webpage

### Collect actors names
#### Because we are building a dataset of actors images, we needed to first collect names of a fair number of actors. In order to do this, I will scrape the first few pages of IMDbs list of top 1000 actors.

In [3]:
# scrapes specified page of imdbs list of top 1000 actors
def collect_actors(url, actors):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    curr_actors = [h3.find('a').text for h3 in soup.find_all('h3', attrs={'class':'lister-item-header'})]   # get the name of each actor and add it to the list parameter
    actors += curr_actors
    return actors

In [4]:
manager = multiprocessing.Manager()
actors = manager.list()   # list of actors names shared between threads
jobs = []
# start 3 process each scraping a different page of the imdbs top 1000 actors list
for i in range(1, actor_num):
    p = multiprocessing.Process(target=collect_actors, args=(imdb_url.format(i), actors))
    jobs.append(p)
    p.start()

# wait for each process to complete
for proc in jobs:
    proc.join()

actors = list(set(actors))   # remove any duplicate names
actors = [a[1:-1] for a in actors]   # remove leading space and trailing newline from each name

### Collect images
#### Now that we have collected the names of the top 300 actors from imdbs top 1000 actors list, we can begin to collect images by scraping Google Images for each actor in the list inorder to build our dataset that will later be used to train a facial recognition cnn.

In [5]:
def fetch_image_urls(query, max_links_to_fetch, wd, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.rg_ic")
        number_results = len(thumbnail_results)
                
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.irc_mi')
            for actual_image in actual_images:
                if actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(1)
            load_more_button = wd.find_element_by_css_selector(".ksb")
            if load_more_button:
                wd.execute_script("document.querySelector('.ksb').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [6]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
        
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [7]:
def search_and_download(search_term, number_images):
    target_folder = '_'.join(search_term.lower().split(' ')) + '/'   # build the target folder path

    # make the path if it does not exist
    if not os.path.exists(target_folder):   
        os.makedirs(target_folder)

    # fetch all the images for the search term
    with webdriver.Chrome('/usr/bin/chromedriver') as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
    
    # save every image collected locally
    for elem in res:
        persist_image(target_folder,elem)
        
    # move each image save to an S3 bucket and remove the local directory and all its contents
    bucket_name = 'actors-images'
    file_list = os.listdir(target_folder)
    for file in file_list:
        s3.upload_file(target_folder + file, bucket_name, target_folder + file)
 
    shutil.rmtree(target_folder)

In [8]:
# break down the number of actors being searched for in chucks of 5
# for i in range(0, len(actors), 5):
for i in range(0, 20, group_size):
    sub_actors = actors[i:i+5]
    jobs = []
    
    # create a process for each search term in the sublist
    for i in range(len(sub_actors)):
        p = multiprocessing.Process(target=search_and_download, args=(sub_actors[i], 10))
        jobs.append(p)
        p.start()
        
    # wait for each process to complete
    for proc in jobs:
        proc.join()

Found: 0 image links, looking for more ...


Process Process-6:
Traceback (most recent call last):
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-02ae8f29c04e>", line 10, in search_and_download
    res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
  File "<ipython-input-5-9c3f88365f2b>", line 44, in fetch_image_urls
    load_more_button = wd.find_element_by_css_selector(".ksb")
  File "/home/connor/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 598, in find_element_by_css_selector
    return self.find_element(by=By.CSS_SELECTOR, value=css_selector)
  File "/home/connor/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 978, in find_element
    'value': value})['value']
  File "/home/connor/anaconda3

Found: 10 image links, done!
Found: 10 image links, done!
Found: 10 image links, done!
Found: 10 image links, done!
SUCCESS - saved https://pbs.twimg.com/profile_images/742877069793742848/c0Ec2mTU.jpg - as samuel_l._jackson/0454eb253e.jpg
SUCCESS - saved https://www.goldderby.com/wp-content/uploads/2018/06/Samuel-L-Jackson-movies-ranked.jpg?w=620&h=360&crop=1 - as samuel_l._jackson/72af50e5ef.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/d/da/Robert_Duvall_2_by_David_Shankbone.jpg - as robert_duvall/dafdf1cce0.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BMTQ1NTQwMTYxNl5BMl5BanBnXkFtZTYwMjA1MzY1._V1_UY1200_CR84,0,630,1200_AL_.jpg - as samuel_l._jackson/c290b0d275.jpg
SUCCESS - saved https://media.npr.org/assets/artslife/arts/2010/06/duvall_wide-6f86dac409a7c696ddb81d15427a1d80bece1a9d-s800-c85.jpg - as robert_duvall/8635f8872e.jpg
SUCCESS - saved https://nyppagesix.files.wordpress.com/2019/03/031619-samuelljackson.jpg?quality=90&strip=all&w=618&h=410&

SUCCESS - saved https://m.media-amazon.com/images/M/MV5BMTEwNDk5MTU2NTNeQTJeQWpwZ15BbWU3MDczNjEzMTM@._V1_UY1200_CR139,0,630,1200_AL_.jpg - as glenn_close/4a4690a74a.jpg
ERROR - Could not save https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=149264868600 - cannot identify image file <_io.BytesIO object at 0x7ffa4b6528e0>
SUCCESS - saved https://cdn1.thr.com/sites/default/files/2017/01/marytylermoore_color_b-h_2017.jpg - as mary_tyler_moore/47b03c5fab.jpg
SUCCESS - saved https://timedotcom.files.wordpress.com/2015/10/jon-voight.jpg - as jon_voight/5f8535e313.jpg
SUCCESS - saved https://images.squarespace-cdn.com/content/v1/55f32473e4b029b54a7228d2/1538520380892-W31KTULZF0YJC6QJYPBC/ke17ZwdGBToddI8pDm48kDFgITcRoterXoQdllT5ciUUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKcV7ZyRJyI8bwZiMJRrgPaAKqUaXS0tb9q_dTyNVba_kClt3J5x-w6oTQbPni4jzRa/Bean+Christoph+Waltz.jpg?format=750w - as christoph_waltz/d5f51605bc.jpg
SUCCESS - saved https://www.indi

SUCCESS - saved https://pmcvariety.files.wordpress.com/2018/05/morgan-freeman-sexual-harassment-7.jpg?w=1000 - as morgan_freeman/2287daede9.jpg
SUCCESS - saved https://i0.wp.com/metro.co.uk/wp-content/uploads/2019/05/SEI_67120354.jpg?quality=90&strip=all&zoom=1&resize=644%2C858&ssl=1 - as doris_day/69f358a2ae.jpg
SUCCESS - saved https://i1.wp.com/metro.co.uk/wp-content/uploads/2019/10/PRI_87751078-e1570000961728.jpg?quality=90&strip=all&zoom=1&resize=644%2C467&ssl=1 - as angelina_jolie/c12f570d4d.jpg
SUCCESS - saved https://static01.nyt.com/images/2014/04/26/arts/Day-Doris-adv-obit-slide-F7E6/Day-Doris-adv-obit-slide-F7E6-articleLarge-v3.jpg?quality=75&auto=webp&disable=upscale - as doris_day/4f30656040.jpg
SUCCESS - saved https://image.insider.com/5d8ce12b6f24eb027370e97c?width=1100&format=jpeg&auto=webp - as angelina_jolie/97450cb553.jpg
SUCCESS - saved https://a57.foxnews.com/media2.foxnews.com/BrightCove/694940094001/2017/07/29/931/524/694940094001_5525670444001_5525659920001-vs.jp

SUCCESS - saved https://images-na.ssl-images-amazon.com/images/I/611DjHu2WLL._SX385_.jpg - as kurt_russell/3bc20988ce.jpg
SUCCESS - saved http://images.fandango.com/ImageRenderer/300/0/redesign/static/img/default_poster.png/0/images/masterrepository/performer%20images/p62232/p62232.jpg - as kurt_russell/96956d4eec.jpg
SUCCESS - saved https://www.biography.com/.image/t_share/MTIwNjA4NjM0MDAwNjcyMjY4/christopher-walken-9521854-1-402.jpg - as christopher_walken/a6c0052359.jpg
SUCCESS - saved https://nyppagesix.files.wordpress.com/2019/06/glenda-jackson.jpg?quality=90&strip=all&w=618&h=410&crop=1 - as glenda_jackson/644bed8a37.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/thumb/a/ad/Kurt_Russell_1974.JPG/220px-Kurt_Russell_1974.JPG - as kurt_russell/e8784d8bf5.jpg
SUCCESS - saved https://i.ytimg.com/vi/9Fqc6_yd_m8/maxresdefault.jpg - as christopher_walken/652e2e990e.jpg
SUCCESS - saved https://www.bfi.org.uk/sites/bfi.org.uk/files/styles/full/public/image/coup-de-torch